<a href="https://colab.research.google.com/github/Mr-Hackrr/pySpark/blob/main/transaction_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install pymupdf pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 36.5 MB/s eta 0:00:00


In [36]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=ed9c9b484753943ad7afa48877d3cfe57ff0ce682cacc86149d04a4f82dd8738
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
import fitz
import pandas as pd

In [37]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [5]:
pdf = fitz.open("transactions.pdf")

In [38]:
spark = SparkSession.builder.appName("transactions").getOrCreate()

In [48]:
# prompt: get all text from document change it text file

doc = fitz.open("transactions.pdf")  # open document
out = open("transactions.txt", "wb")  # open text output
for page in doc:  # iterate the document pages
    text = page.get_text().encode("utf8")  # get plain text (bytestring)
    out.write(text)  # write text of page
    out.write(bytes((12,)))  # write page delimiter (form feed 0x0C)
out.close()


In [61]:
df = spark.read.text("transactions.txt")

In [62]:
df.show()

+--------------------+
|               value|
+--------------------+
|                    |
|                Name|
|SRIHARIKOTA GOVAR...|
|             Address|
|SO SRINIVASULU D ...|
|SATYANARAYANA PURAM |
| MAYPAD ROAD NELLORE|
|                City|
|             NELLORE|
|               State|
|      ANDHRA PRADESH|
|             Pincode|
|              524002|
|           Mobile No|
|        919381795961|
|            Email Id|
|         Home branch|
|       STONEHOUSEPET|
|                IFSC|
|         UBIN0805173|
+--------------------+
only showing top 20 rows



In [63]:
# prompt: no of rowa in data

df.count()


5515

In [64]:
from pyspark.sql.window import Window

In [22]:
Columns = ['S.No','Date','Transaction Id','Remarks','Amount(Rs.)','Balance(Rs.)',]

In [65]:
df = df.withColumn('index',f.row_number().over(Window.orderBy(f.lit(1))))
df.show()

+--------------------+-----+
|               value|index|
+--------------------+-----+
|                    |    1|
|                Name|    2|
|SRIHARIKOTA GOVAR...|    3|
|             Address|    4|
|SO SRINIVASULU D ...|    5|
|SATYANARAYANA PURAM |    6|
| MAYPAD ROAD NELLORE|    7|
|                City|    8|
|             NELLORE|    9|
|               State|   10|
|      ANDHRA PRADESH|   11|
|             Pincode|   12|
|              524002|   13|
|           Mobile No|   14|
|        919381795961|   15|
|            Email Id|   16|
|         Home branch|   17|
|       STONEHOUSEPET|   18|
|                IFSC|   19|
|         UBIN0805173|   20|
+--------------------+-----+
only showing top 20 rows



In [71]:
df = df.filter(df.index > 37)

In [72]:
df.show(50)

+--------------------+-----+
|               value|index|
+--------------------+-----+
|                S.No|   38|
|                Date|   39|
|      Transaction Id|   40|
|             Remarks|   41|
|         Amount(Rs.)|   42|
|        Balance(Rs.)|   43|
|                   1|   44|
|          28/07/2024|   45|
|           S80067750|   46|
|UPIAR/45765105325...|   47|
|       Q090933256@yb|   48|
|          40.00 (Dr)|   49|
|         265.31 (Cr)|   50|
|                   2|   51|
|          28/07/2024|   52|
|           S77556410|   53|
|UPIAR/45769338313...|   54|
|       9347857991@ax|   55|
|         500.00 (Dr)|   56|
|         305.31 (Cr)|   57|
|                   3|   58|
|          28/07/2024|   59|
|           S77531041|   60|
|UPIAR/45767039326...|   61|
|       8790354176@yb|   62|
|         325.00 (Dr)|   63|
|         805.31 (Cr)|   64|
|                   4|   65|
|          28/07/2024|   66|
|           S68849599|   67|
|NEFT:SRIHARIKOTA ...|   68|
|    N21024317

In [ ]:
df.withColumn("Amount", f.regexp_extract('value',r""))